<a href="https://colab.research.google.com/github/mKozik7/Data_Analytics/blob/main/Nba2k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [28]:
df = pd.read_csv('nba2k.csv', index_col = False)
print(df[['full_name','salary','rating','position']])

                 full_name     salary  rating position
0             LeBron James  $37436858      97        F
1            Kawhi Leonard  $32742000      97        F
2    Giannis Antetokounmpo  $25842697      96      F-G
3             Kevin Durant  $37199000      96        F
4             James Harden  $38199000      96        G
..                     ...        ...     ...      ...
434           Kevin Durant  $40918900      95        F
435          Anthony Davis  $35361360      95      F-C
436            Luka Doncic  $10174391      94      G-F
437         Damian Lillard  $39344900      94        G
438            Joel Embiid  $31579390      91        C

[439 rows x 4 columns]


In [29]:
initial_population = []
positions = ['F','F-G','C','G','G-F','C-F','F-C']


In [42]:
def initial_population(data,n, positions):
  init_population = []
  for i in range(n):
    team = []
    for j in positions:
      player = data[data['position'] == j].sample(1)
      team.append(player.iloc[0].full_name)
    init_population.append(team)
  return init_population
  
target_pos = ['F','F-G','C','G','G-F','C-F','F-C']
n = 50
data = df
population = initial_population(data, n, target_pos)

[['Ryan Broekhoff', 'Justin Jackson', 'Pau Gasol', 'Frank Ntilikina', 'Andre Roberson', 'Al Horford', 'Tyler Lydon'], ['Omari Spellman', 'Gerald Green', 'Meyers Leonard', 'Shaquille Harrison', 'CJ Miles', 'Jakob Poeltl', 'Mike Muscala'], ['Tristan Thompson', 'Mario Hezonja', 'Willie Cauley-Stein', 'Michael Frazier', 'Kevin Porter', 'Skal Labissiere', 'Bam Adebayo'], ['Moritz Wagner', 'Cedi Osman', 'Pau Gasol', 'Goran Dragic', 'Bojan Bogdanovic', 'Mason Plumlee', 'Frank Kaminsky'], ['Harrison Barnes', 'Giannis Antetokounmpo', 'Jusuf Nurkic', 'Jrue Holiday', 'DeMar DeRozan', 'Mason Plumlee', 'Tony Bradley'], ['Otto Porter', 'Gerald Green', 'Mo Bamba', 'Sviatoslav Mykhailiuk', 'Tyrone Wallace', 'Cody Zeller', 'Harry Giles'], ['Malcolm Miller', 'Cedi Osman', 'Luke Kornet', 'Shaquille Harrison', 'Luka Doncic', 'Mason Plumlee', 'TJ Leaf'], ['Doug McDermott', 'Justin Jackson', 'Steven Adams', 'Alec Burks', 'Kenrich Williams', 'Jakob Poeltl', 'Thomas Bryant'], ["Royce O'Neale", 'Troy Brown', '

In [37]:
def crossover(pop1, pop2):
    if random.random() < probability_crossover:
        pos = random.randint(0,len(pop1))
        crossed_pop1 = pop1[:pos] + pop2[pos:]
        crossed_pop2 = pop2[:pos] + pop1[pos:]
        return [crossed_pop1, crossed_pop2]

    else:
        return [pop1,pop2]
probability_crossover = 0.85


In [39]:
parents = []
for i in  range(0, n - 1, 2):
  indiv_1= population[i]
  indiv_2 = population[i+1]
  parents.extend(crossover(indiv_1, indiv_2))


[['Pascal Siakam', 'Justin Jackson', 'Kelly Olynyk', 'Yogi Ferrell', 'Kevin Porter', 'Willy Hernangomez', 'Maxi Kleber'], ['Michael Porter', 'Nemanja Bjelica', 'Naz Reid', 'Dennis Schroeder', 'Antonio Blakeney', 'Cody Zeller', 'Tony Bradley'], ['Glenn Robinson III', 'Justin Jackson', 'Jusuf Nurkic', 'Monte Morris', 'Andre Roberson', 'Cody Zeller', 'Henry Ellenson'], ['Dario Saric', 'Giannis Antetokounmpo', 'Nikola Vucevic', 'Terrance Ferguson', 'CJ Miles', 'Skal Labissiere', 'Karl-Anthony Towns'], ['PJ Tucker', 'Nicolas Batum', 'Nikola Vucevic', 'Bryn Forbes', 'Luka Doncic', 'Skal Labissiere', 'Amile Jefferson'], ['Alfonzo McKinnie', 'Justin Jackson', 'Nikola Jokic', 'Caris LeVert', 'Andre Roberson', 'Mason Plumlee', 'Maxi Kleber'], ['James Ennis', 'Nicolas Batum', 'Deandre Ayton', 'Delon Wright', 'Kevin Porter', 'Willy Hernangomez', 'Thomas Bryant'], ['Duncan Robinson', 'Giannis Antetokounmpo', 'Daniel Theis', 'Matt Thomas', 'Allonzo Trier', 'Cody Zeller', 'Yante Maten'], ['Abdel Nade